In [1]:
!pip install sentence_transformers
!pip install pyterrier
!pip install Arabic-Stopwords
!pip install pytrec_eval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.5/360.5 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.4/126.4 kB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pytrec_eval: filename=pytrec_eval-0.5-cp310-cp310-linux_x86_64.whl size=308218 sha256=319e0d81d056c35338e42ad41c8285df30a836620a6989ca5cf0b986664f4a1c
  Stored in directory: /root/.cache/pip/wheels/51/3a/cd/dcc1ddfc763987d5cb237165d8ac249aa98a23ab90f67317a8
Successfully built pytrec_eval


In [3]:
import numpy as np
import pandas as pd
import re
import pandas as pd
from snowballstemmer import stemmer
import arabicstopwords.arabicstopwords as ar_stp
import pyterrier as pt
import json
from sentence_transformers import SentenceTransformer, InputExample, losses, models, datasets,  CrossEncoder, util
from transformers import AutoTokenizer
import gzip
import os
import torch

if not torch.cuda.is_available():
    print("Warning: No GPU found. Please add GPU to your notebook")

from sklearn.feature_extraction import _stop_words
import string
from tqdm.autonotebook import tqdm

from torch import nn
import os
import time

In [5]:
# The train data for Quran QA 2023
# I already matched the Questions with Passages (Texts)
df = pd.read_csv("/content/data/train_data.csv")
df.head(50)

Question  \
0   اتفاقية ""سيداو"" تقول الرجل لا يملك الولاية ع...   
1   اتفاقية ""سيداو"" تقول الرجل لا يملك الولاية ع...   
2   اتفاقية ""سيداو"" تقول الرجل لا يملك الولاية ع...   
3   اتفاقية ""سيداو"" تقول الرجل لا يملك الولاية ع...   
4   اتفاقية ""سيداو"" تقول الرجل لا يملك الولاية ع...   
5   اتفاقية ""سيداو"" تقول لا يُسمح للرجل بتعدد ال...   
6   اتهم القرآن بأنه السبب في الدكتاتورية الإسلامي...   
7   اتهم القرآن بأنه السبب في الدكتاتورية الإسلامي...   
8   اتهم القرآن بأنه السبب في الدكتاتورية الإسلامي...   
9   اتهم القرآن بأنه السبب في الدكتاتورية الإسلامي...   
10  اتهم القرآن بأنه السبب في الدكتاتورية الإسلامي...   
11  اتهم القرآن بأنه السبب في الدكتاتورية الإسلامي...   
12  اتهم القرآن بأنه السبب في الدكتاتورية الإسلامي...   
13  اتهم القرآن بأنه السبب في الدكتاتورية الإسلامي...   
14  إذا كان الشيطان شراً فلماذا خلقه الله وسمح له ...   
15  إذا كان الشيطان شراً فلماذا خلقه الله وسمح له ...   
16  إذا كان الشيطان شراً فلماذا خلقه الله وسمح له ...   
17  إذا كان الشيطان شراً فلماذا خلقه الله وسمح له ...   
18  الحلي من الذهب لا يجوز أن يلبسها الرجل في الدن...   
19  الحلي من الذهب لا يجوز أن يلبسها الرجل في الدن...   
20  الحلي من الذهب لا يجوز أن يلبسها الرجل في الدن...   
21  المصلح هو الصالح في ذاته والمصلح لغيره. ما الد...   
22  المصلح هو الصالح في ذاته والمصلح لغيره. ما الد...   
23  المصلح هو الصالح في ذاته والمصلح لغيره. ما الد...   
24  المصلح هو الصالح في ذاته والمصلح لغيره. ما الد...   
25  المصلح هو الصالح في ذاته والمصلح لغيره. ما الد...   
26  المصلح هو الصالح في ذاته والمصلح لغيره. ما الد...   
27  المصلح هو الصالح في ذاته والمصلح لغيره. ما الد...   
28  المصلح هو الصالح في ذاته والمصلح لغيره. ما الد...   
29  المصلح هو الصالح في ذاته والمصلح لغيره. ما الد...   
30        إن كان الله قدّر علي أفعالي فلماذا يحاسبني؟   
31        إن كان الله قدّر علي أفعالي فلماذا يحاسبني؟   
32        إن كان الله قدّر علي أفعالي فلماذا يحاسبني؟   
33        إن كان الله قدّر علي أفعالي فلماذا يحاسبني؟   
34        إن كان الله قدّر علي أفعالي فلماذا يحاسبني؟   
35        إن كان الله قدّر علي أفعالي فلماذا يحاسبني؟   
36        إن كان الله قدّر علي أفعالي فلماذا يحاسبني؟   
37        إن كان الله قدّر علي أفعالي فلماذا يحاسبني؟   
38        إن كان الله قدّر علي أفعالي فلماذا يحاسبني؟   
39        إن كان الله قدّر علي أفعالي فلماذا يحاسبني؟   
40        إن كان الله قدّر علي أفعالي فلماذا يحاسبني؟   
41        إن كان الله قدّر علي أفعالي فلماذا يحاسبني؟   
42        إن كان الله قدّر علي أفعالي فلماذا يحاسبني؟   
43        إن كان الله قدّر علي أفعالي فلماذا يحاسبني؟   
44        إن كان الله قدّر علي أفعالي فلماذا يحاسبني؟   
45        إن كان الله قدّر علي أفعالي فلماذا يحاسبني؟   
46        إن كان الله قدّر علي أفعالي فلماذا يحاسبني؟   
47        إن كان الله قدّر علي أفعالي فلماذا يحاسبني؟   
48        إن كان الله قدّر علي أفعالي فلماذا يحاسبني؟   
49        إن كان الله قدّر علي أفعالي فلماذا يحاسبني؟   

                                              Passage  
0   الرجال قوامون على النساء بما فضل الله بعضهم عل...  
1   والذين يتوفون منكم ويذرون أزواجا يتربصن بأنفسه...  
2   والمطلقات يتربصن بأنفسهن ثلاثة قروء ولا يحل له...  
3   يا أيها الذين آمنوا قوا أنفسكم وأهليكم نارا وق...  
4   يا أيها النبي قل لأزواجك وبناتك ونساء المؤمنين...  
5   وآتوا اليتامى أموالهم ولا تتبدلوا الخبيث بالطي...  
6   إن هذه تذكرة فمن شاء اتخذ إلى ربه سبيلا. وما ت...  
7   إنا أنزلنا عليك الكتاب للناس بالحق فمن اهتدى ف...  
8   فما لهم عن التذكرة معرضين. كأنهم حمر مستنفرة. ...  
9   قل يا أيها الكافرون. لا أعبد ما تعبدون. ولا أن...  
10  لا إكراه في الدين قد تبين الرشد من الغي فمن يك...  
11  من جاء بالحسنة فله خير منها وهم من فزع يومئذ آ...  
12  وقل الحق من ربكم فمن شاء فليؤمن ومن شاء فليكفر...  
13  يوم يقوم الروح والملائكة صفا لا يتكلمون إلا من...  
14  أولم ير الذين كفروا أن السماوات والأرض كانتا ر...  
15  ولقد صدق عليهم إبليس ظنه فاتبعوه إلا فريقا من ...  
16  وما أرسلنا من قبلك من رسول ولا نبي إلا إذا تمن...  
17  يا أيها الناس إن وعد الله حق فلا تغرنكم الحياة...  
18  إن الذين يتلون كتاب الله وأقاموا الصلاة وأنفقو...  
19  إن الله يدخل 

In [6]:
len(df)

1522

In [7]:
df2 = pd.read_csv("/content/data/QA_2022.csv")
df2.head()

pq_id                                            passage  surah  \
0     2:8-16_364  ومن الناس من يقول آمنا بالله وباليوم الآخر وما...      2   
1  2:174-176_364  إن الذين يكتمون ما أنزل الله من الكتاب ويشترون...      2   
2   14:47-52_364  فلا تحسبن الله مخلف وعده رسله إن الله عزيز ذو ...     14   
3   17:12-17_364  وجعلنا الليل والنهار آيتين فمحونا آية الليل وج...     17   
4   18:29-31_364  وقل الحق من ربكم فمن شاء فليؤمن ومن شاء فليكفر...     18   

    verses                                          questions  \
0   16-Aug  لماذا سيُحاسب ويُعذب الضال يوم القيامة ان كان ...   
1  174-176  لماذا سيُحاسب ويُعذب الضال يوم القيامة ان كان ...   
2    47-52  لماذا سيُحاسب ويُعذب الضال يوم القيامة ان كان ...   
3   17-Dec  لماذا سيُحاسب ويُعذب الضال يوم القيامة ان كان ...   
4    29-31  لماذا سيُحاسب ويُعذب الضال يوم القيامة ان كان ...   

                                             answers  
0  [{'text': 'أولئك الذين اشتروا الضلالة بالهدى',...  
1  [{'text': 'أولئك الذين اشتروا الضلالة بالهدى و...  
2  [{'text': 'ليجزي الله كل نفس ما كسبت', 'start_...  
3  [{'text': 'كل إنسان ألزمناه طائره في عنقه', 's...  
4  [{'text': 'من شاء فليؤمن ومن شاء فليكفر', 'sta...

In [8]:
train_examples = []
for n in np.arange(0, len(df)):
  train_examples.append(InputExample(texts=[str(df['Question'][n]), str(df['Passage'][n])]))

for n in np.arange(0, len(df2)):
  train_examples.append(InputExample(texts=[str(df2['questions'][n]), str(df2['answers'][n])]))

# For the MultipleNegativesRankingLoss, we use NoDuplicatesDataLoader to make sure no duplicated records
train_dataloader = datasets.NoDuplicatesDataLoader(train_examples, batch_size=16)

In [9]:
! git clone https://gitlab.com/bigirqu/quran-qa-2023.git

Cloning into 'quran-qa-2023'...
remote: Enumerating objects: 1018, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 1018 (delta 32), reused 0 (delta 0), pack-reused 963
Receiving objects: 100% (1018/1018), 1.14 MiB | 2.89 MiB/s, done.
Resolving deltas: 100% (479/479), done.


In [11]:
df_validation = pd.read_csv("/content/quran-qa-2023/Task-A/data/QQA23_TaskA_ayatec_v1.2_test.tsv", sep = '\t', names = ['ID_Question', 'Question'])
df_validation.head()

ID_Question                                           Question
0          500      من ماذا نجى الله سبحانه وتعالى سيدنا إبراهيم؟
1          502  ما الدلائل على أن الانبياء والرسل لا يعلمون ال...
2          504            أين يقع الكهف الذي أوى إليه فتية الكهف؟
3          505  ما الدلائل على أن القرآن أُنزل لجميع الناس أو ...
4          508  مع ازدياد عدد سكان الارض، هل يجب أن نقلق أو نخ...

In [12]:
questions_list = df_validation['ID_Question'].tolist()

In [13]:
df_val = pd.read_csv("/content/quran-qa-2023/Task-A/data/Thematic_QPC/QQA23_TaskA_QPC_v1.1.tsv", sep='\t', names=['ID', 'Passage'])
df_val.head()


ID                                            Passage
0  1:1-4  بسم الله الرحمن الرحيم. الحمد لله رب العالمين....
1  1:5-6     إياك نعبد وإياك نستعين. اهدنا الصراط المستقيم.
2  1:7-7  صراط الذين أنعمت عليهم غير المغضوب عليهم ولا ا...
3  2:1-2            الم. ذلك الكتاب لا ريب فيه هدى للمتقين.
4  2:3-5  الذين يؤمنون بالغيب ويقيمون الصلاة ومما رزقناه...

In [14]:
passages_val = []
for n in df_val['Passage']:
  # print(n)
  passages_val.append(str(n))

print(len(passages_val))

1266


In [16]:
bi_encoder = SentenceTransformer('asafaya/bert-base-arabic')
bi_encoder.max_seq_length = 512
top_k = 32

# We use a cross-encoder, to re-rank the bi-encoder's results list to improve the quality
cross_encoder = CrossEncoder('nreimers/mmarco-mMiniLMv2-L12-H384-v1')

config.json:   0%|          | 0.00/813 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [17]:
# Set up the model
word_emb = models.Transformer('Damith/AraELECTRA-discriminator-QuranQA', max_seq_length=512)
pooling = models.Pooling(word_emb.get_word_embedding_dimension())
model = SentenceTransformer(modules=[word_emb, pooling])

# Train loss
train_loss = losses.MultipleNegativesRankingLoss(model)

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/539M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/450 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/761k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [18]:
num_epochs = 8
warmup_steps = int(len(train_dataloader) * num_epochs * 0.1)
model.fit(train_objectives=[(train_dataloader, train_loss)],
          epochs=num_epochs,
          warmup_steps=warmup_steps,
          show_progress_bar=True)

Epoch:   0%|          | 0/8 [00:00<?, ?it/s]

Iteration:   0%|          | 0/163 [00:00<?, ?it/s]

Iteration:   0%|          | 0/163 [00:00<?, ?it/s]

Iteration:   0%|          | 0/163 [00:00<?, ?it/s]

Iteration:   0%|          | 0/163 [00:00<?, ?it/s]

Iteration:   0%|          | 0/163 [00:00<?, ?it/s]

Iteration:   0%|          | 0/163 [00:00<?, ?it/s]

Iteration:   0%|          | 0/163 [00:00<?, ?it/s]

Iteration:   0%|          | 0/163 [00:00<?, ?it/s]

In [19]:
model.save('AraELECTRA.h5')

In [21]:
model = SentenceTransformer('AraELECTRA.h5')

In [22]:
corpus_embeddings = model.encode(passages_val, convert_to_tensor=True, show_progress_bar=True)


Batches:   0%|          | 0/40 [00:00<?, ?it/s]

In [23]:
ar_stemmer = stemmer("arabic")


# normalize the arabic text
def normalize_arabic(text):
    text = re.sub("[إأٱآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    # text = re.sub("ة", "ه", text)
    return(text)

def deNoise(text):
    noise = re.compile(""" ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)
    text = re.sub(noise, '', text)
    return text

def arabic_preprocessing(text):
  text = normalize_arabic(text)
  text = deNoise(text)
  return text

In [26]:
# Example
x = df_validation['Question'][2]
print(x)
arabic_preprocessing(x)

أين يقع الكهف الذي أوى إليه فتية الكهف؟


'اين يقع الكهف الذي اوي اليه فتية الكهف؟'

In [27]:
def search(num):

    questions = []

    answers_bi = []
    score_bi = []
    passage_bi = []
    rank_bi = []
    count_bi = 1


    ##### get the question from index
    query = df_validation.loc[df_validation['ID_Question'] == num, 'Question'].iloc[0]
    query = arabic_preprocessing(query)
    print("Input question:", query)

    ##### Sematic Search #####
    # Encode the query using the bi-encoder and find potentially relevant passages
    question_embedding = model.encode(query, convert_to_tensor=True)
    question_embedding = question_embedding.cuda()
    hits = util.semantic_search(question_embedding, corpus_embeddings, top_k=top_k)
    hits = hits[0]  # Get the hits for the first query

    # Get the top 20 answers
    hits = sorted(hits, key=lambda x: x['score'], reverse=True)
    for hit in hits[0:20]:
      questions.append(query)
      answers_bi.append(passages_val[hit['corpus_id']])
      score_bi.append(hit['score'])
      rank_bi.append(count_bi)
      count_bi += 1
      if passages_val[hit['corpus_id']] == 'nan':
        passage_bi.append('-1')
      else:
        passage_id_val = df_val.loc[df_val['Passage'] == passages_val[hit['corpus_id']], 'ID'].iloc[0]
        passage_bi.append(passage_id_val)


    ##### Save it to a dafataframe #####
    df_answers = pd.DataFrame({
        'Question': questions,
        'answers_bi': answers_bi,
        'rank_bi': rank_bi,
        'score_bi': score_bi,
        'passage_bi': passage_bi
    })

    df_answers['Q0'] = 'Q0'
    df_answers['q_id'] = num

    return df_answers


In [42]:
testing_results = search(500)
testing_results

Input question: من ماذا نجي الله سبحانه وتعالي سيدنا ابراهيم؟


Question  \
0   من ماذا نجي الله سبحانه وتعالي سيدنا ابراهيم؟   
1   من ماذا نجي الله سبحانه وتعالي سيدنا ابراهيم؟   
2   من ماذا نجي الله سبحانه وتعالي سيدنا ابراهيم؟   
3   من ماذا نجي الله سبحانه وتعالي سيدنا ابراهيم؟   
4   من ماذا نجي الله سبحانه وتعالي سيدنا ابراهيم؟   
5   من ماذا نجي الله سبحانه وتعالي سيدنا ابراهيم؟   
6   من ماذا نجي الله سبحانه وتعالي سيدنا ابراهيم؟   
7   من ماذا نجي الله سبحانه وتعالي سيدنا ابراهيم؟   
8   من ماذا نجي الله سبحانه وتعالي سيدنا ابراهيم؟   
9   من ماذا نجي الله سبحانه وتعالي سيدنا ابراهيم؟   
10  من ماذا نجي الله سبحانه وتعالي سيدنا ابراهيم؟   
11  من ماذا نجي الله سبحانه وتعالي سيدنا ابراهيم؟   
12  من ماذا نجي الله سبحانه وتعالي سيدنا ابراهيم؟   
13  من ماذا نجي الله سبحانه وتعالي سيدنا ابراهيم؟   
14  من ماذا نجي الله سبحانه وتعالي سيدنا ابراهيم؟   
15  من ماذا نجي الله سبحانه وتعالي سيدنا ابراهيم؟   
16  من ماذا نجي الله سبحانه وتعالي سيدنا ابراهيم؟   
17  من ماذا نجي الله سبحانه وتعالي سيدنا ابراهيم؟   
18  من ماذا نجي الله سبحانه وتعالي سيدنا ابراهيم؟   
19  من ماذا نجي الله سبحانه وتعالي سيدنا ابراهيم؟   

                                           answers_bi  rank_bi  score_bi  \
0   فما كان جواب قومه إلا أن قالوا اقتلوه أو حرقوه...        1  0.750652   
1   وإذ قلنا للملائكة اسجدوا لآدم فسجدوا إلا إبليس...        2  0.740425   
2   ولقد آتينا إبراهيم رشده من قبل وكنا به عالمين....        3  0.735212   
3   وقال فرعون ائتوني بكل ساحر عليم. فلما جاء السح...        4  0.725035   
4   ومكروا ومكر الله والله خير الماكرين. إذ قال ال...        5  0.724141   
5   وإلى عاد أخاهم هودا قال يا قوم اعبدوا الله ما ...        6  0.715058   
6   وإذ قلنا للملائكة اسجدوا لآدم فسجدوا إلا إبليس...        7  0.713177   
7   وإذ قلنا للملائكة اسجدوا لآدم فسجدوا إلا إبليس...        8  0.701404   
8   وهل أتاك نبأ الخصم إذ تسوروا المحراب. إذ دخلوا...        9  0.700146   
9   ضرب الله مثلا للذين كفروا امرأت نوح وامرأت لوط...       10  0.698503   
10  كذبت قوم نوح المرسلين. إذ قال لهم أخوهم نوح أل...       11  0.695612   
11  وراودته التي هو في بيتها عن نفسه وغلقت الأبواب...       12  0.695297   
12  واذكر في الكتاب إبراهيم إنه كان صديقا نبيا. إذ...       13  0.693332   
13  ولما بلغ أشده واستوى آتيناه حكما وعلما وكذلك ن...       14  0.690524   
14  كذب أصحاب الأيكة المرسلين. إذ قال لهم شعيب ألا...       15  0.689067   
15  ولقد خلقنا الإنسان من صلصال من حمإ مسنون. والج...       16  0.683815   
16  قلنا يا نار كوني بردا وسلاما على إبراهيم. وأرا...       17  0.676540   
17  وإذ قال إبراهيم لأبيه آزر أتتخذ أصناما آلهة إن...       18  0.673095   
18  وإلى عاد أخاهم هودا قال يا قوم اعبدوا الله ما ...       19  0.667675   
19  ولقد عهدنا إلى آدم من قبل فنسي ولم نجد له عزما...       20  0.658502   

    passage_bi  Q0  q_id  
0     29:24-27  Q0   500  
1      2:34-39  Q0   500  
2     21:51-68  Q0   500  
3     10:79-86  Q0   500  
4      3:54-58  Q0   500  
5     11:50-60  Q0   500  
6     18:50-53  Q0   500  
7     17:61-65  Q0   500  
8     38:21-26  Q0   500  
9     66:10-12  Q0   500  
10  26:105-122  Q0   500  
11    12:23-29  Q0   500  
12    19:41-50  Q0   500  
13    28:14-19  Q0   500  
14  26:176-191  Q0   500  
15    15:26-44  Q0   500  
16    21:69-73  Q0   500  
17     6:74-79  Q0   500  
18     7:65-72  Q0   500  
19  20:115-122  Q0   500

In [43]:
for q in questions_list:
  if q == 500:
    print(q)
    df_results = search(q)
    df_results_merge = pd.concat([df_results])
    continue
  else:
    print(q)
    df_results_again = search(q)
    df_results_merge = pd.concat([df_results_merge, df_results_again])

500
Input question: من ماذا نجي الله سبحانه وتعالي سيدنا ابراهيم؟
502
Input question: ما الدلاءل علي ان الانبياء والرسل لا يعلمون الغيب؟
504
Input question: اين يقع الكهف الذي اوي اليه فتية الكهف؟
505
Input question: ما الدلاءل علي ان القران انزل لجميع الناس او جميع الامم والشعوب؟
508
Input question: مع ازدياد عدد سكان الارض، هل يجب ان نقلق او نخاف من عدم كفاية مصادر الغذاء والرزق؟
510
Input question: اتفاقية ""سيداو"" تقول لا يسمح للرجل بتعدد الزوجات. كيف نرد علي ذلك من نصوص القران؟
513
Input question: اتفاقية ""سيداو"" تقول الرجل لا يملك الولاية علي المراة والاب لا يملك الولاية علي بناته.  كيف نرد علي ذلك من نصوص القران؟
518
Input question: لكون البخل لا يقتصر علي المال فقط، ماذا يشمل البخل او الشح (بما فيه المال)؟
519
Input question: ما هي عواقب البخل او الشح؟
521
Input question: من المشككين من يتساءل: هل عوقب جميع البشر علي ذنب اقترفه سيدنا ادم؟
522
Input question: ما الفرق بين التبذير والاسراف؟
525
Input question: ماذا تشمل الامانة في القران؟
527
Input question: لماذا قد يبتلي الل

In [44]:
df_results_merge.reset_index(inplace=True)

In [45]:
df_results_bi = df_results_merge[['q_id', 'Q0', 'passage_bi', 'rank_bi', 'score_bi','Question','answers_bi']]

In [46]:
df_results_bi['run'] = 'LKAU23_run60_AraELECTRA'
df_results_bi

q_id  Q0  passage_bi  rank_bi  score_bi  \
0      500  Q0    29:24-27        1  0.750652   
1      500  Q0     2:34-39        2  0.740425   
2      500  Q0    21:51-68        3  0.735212   
3      500  Q0    10:79-86        4  0.725035   
4      500  Q0     3:54-58        5  0.724141   
...    ...  ..         ...      ...       ...   
1035   613  Q0    22:14-15       16  0.785964   
1036   613  Q0    29:56-60       17  0.784790   
1037   613  Q0    36:20-27       18  0.783031   
1038   613  Q0  11:103-108       19  0.779976   
1039   613  Q0    16:30-32       20  0.778267   

                                           Question  \
0     من ماذا نجي الله سبحانه وتعالي سيدنا ابراهيم؟   
1     من ماذا نجي الله سبحانه وتعالي سيدنا ابراهيم؟   
2     من ماذا نجي الله سبحانه وتعالي سيدنا ابراهيم؟   
3     من ماذا نجي الله سبحانه وتعالي سيدنا ابراهيم؟   
4     من ماذا نجي الله سبحانه وتعالي سيدنا ابراهيم؟   
...                                             ...   
1035        كيف نوفق بين الخوف من الله والرجاء فيه؟   
1036        كيف نوفق بين الخوف من الله والرجاء فيه؟   
1037        كيف نوفق بين الخوف من الله والرجاء فيه؟   
1038        كيف نوفق بين الخوف من الله والرجاء فيه؟   
1039        كيف نوفق بين الخوف من الله والرجاء فيه؟   

                                             answers_bi  \
0     فما كان جواب قومه إلا أن قالوا اقتلوه أو حرقوه...   
1     وإذ قلنا للملائكة اسجدوا لآدم فسجدوا إلا إبليس...   
2     ولقد آتينا إبراهيم رشده من قبل وكنا به عالمين....   
3     وقال فرعون ائتوني بكل ساحر عليم. فلما جاء السح...   
4     ومكروا ومكر الله والله خير الماكرين. إذ قال ال...   
...                                                 ...   
1035  إن الله يدخل الذين آمنوا وعملوا الصالحات جنات ...   
1036  يا عبادي الذين آمنوا إن أرضي واسعة فإياي فاعبد...   
1037  وجاء من أقصى المدينة رجل يسعى قال يا قوم اتبعو...   
1038  إن في ذلك لآية لمن خاف عذاب الآخرة ذلك يوم مجم...   
1039  وقيل للذين اتقوا ماذا أنزل ربكم قالوا خيرا للذ...   

                          run  
0     LKAU23_run60_AraELECTRA  
1     LKAU23_run60_AraELECTRA  
2     LKAU23_run60_AraELECTRA  
3     LKAU23_run60_AraELECTRA  
4     LKAU23_run60_AraELECTRA  
...                       ...  
1035  LKAU23_run60_AraELECTRA  
1036  LKAU23_run60_AraELECTRA  
1037  LKAU23_run60_AraELECTRA  
1038  LKAU23_run60_AraELECTRA  
1039  LKAU23_run60_AraELECTRA  

[1040 rows x 8 columns]

In [47]:
df_results_bi.to_csv("LKAU23_run60_AraELECTRA.tsv", sep='\t', index=False, header=False)


In [52]:
df_results_bi = df_results_merge[['q_id', 'Q0', 'passage_bi', 'rank_bi', 'score_bi']]

In [53]:
df_results_bi['run'] = 'bigIR_run02.tsv'
df_results_bi

<ipython-input-53-002f8f9b3f92>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_results_bi['run'] = 'bigIR_run02.tsv'


q_id  Q0  passage_bi  rank_bi  score_bi              run
0      500  Q0    29:24-27        1  0.750652  bigIR_run02.tsv
1      500  Q0     2:34-39        2  0.740425  bigIR_run02.tsv
2      500  Q0    21:51-68        3  0.735212  bigIR_run02.tsv
3      500  Q0    10:79-86        4  0.725035  bigIR_run02.tsv
4      500  Q0     3:54-58        5  0.724141  bigIR_run02.tsv
...    ...  ..         ...      ...       ...              ...
1035   613  Q0    22:14-15       16  0.785964  bigIR_run02.tsv
1036   613  Q0    29:56-60       17  0.784790  bigIR_run02.tsv
1037   613  Q0    36:20-27       18  0.783031  bigIR_run02.tsv
1038   613  Q0  11:103-108       19  0.779976  bigIR_run02.tsv
1039   613  Q0    16:30-32       20  0.778267  bigIR_run02.tsv

[1040 rows x 6 columns]

In [54]:
df_results_bi.to_csv("bigIR_run02.tsv", sep='\t', index=False, header=False)

In [56]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [57]:
import shutil
shutil.make_archive('TaskAOutput', 'zip', '/content/AraELECTRA.h5')

'/content/TaskAOutput.zip'

In [58]:
shutil.move('TaskAOutput.zip', '/content/drive/My Drive/AraElectraDiscrminator')

'/content/drive/My Drive/AraElectraDiscrminator'